In [ ]:
import sys
!{sys.executable} -m pip install git+https://github.com/arendjan/tmx-pico-aio.git@sensors
!{sys.executable} -m pip install nest_asyncio

Hardware setup:

You'll need 2 breadboards to fit the larger size ADC module. Might need to cut off one of the power rails.

![button](images/load_cell_bb.png)

Pico connections:

|Pin # | type  | connection |
|-------|------|------------|
| 18   | GND   |         |
| 19   | GP14  | SCK, clock output    |
| 20   | GP15  | DT, data input      |
| 36   | 3V3   |          |

Load cell connection:
|Color | ADC name  | type |
|-------|------|------------|
| Red   | E+   | Excitation +        |
| Black  | E-  |    Excitation -  |
| White| A-  | Input A-   |
| Green| A+   |    Input A+      |

The data received will be the difference between A- and A+. 



In [ ]:
import asyncio
import sys
import time
from tmx_pico_aio import tmx_pico_aio
import nest_asyncio
import struct
nest_asyncio.apply() # for jupyter notebook asyncio stuff

loop = asyncio.get_event_loop()

board = tmx_pico_aio.TmxPicoAio(loop=loop)

Add your callback, like printing or adding it to a list

In [ ]:
async def the_callback(data):
    # print(data)
    [value] = list(struct.unpack('<i', b''.join(list(map(lambda i:i.to_bytes(1, 'big'), data)))))
    print(f"val: {str(value): >8}")

Add the sensor to the system

In [ ]:
async def addSensor(board):
    await asyncio.sleep(1)
    sck = 14
    dt = 15
    # note: this sensor does not use i2c!
    await board.sensors.add_hx711(out=dt,sck=sck,callback=the_callback)

loop.run_until_complete(addSensor(board))



Run for some time

In [ ]:
async def run_till_end(board):
    # when to stop?
    await asyncio.sleep(10)
    await board.shutdown() # Otherwise next start will give you errors and you need to restart the kernel
    await asyncio.sleep(1)
    print("done")

loop.run_until_complete(run_till_end(board))